## Experiment logger
The logger stores experimental data in a single SQLite database. It is intended to be fast and lightweight, but record all necessary meta data and timestamps for experimental trials.

As a consequence, [SQLiteBrowser](http://sqlitebrowser.org/) can be used to browse the log results without having to do any coding.

Most of the entries are stored as JSON strings in the database tables; any object that can be serialised by Python's `json` module can be added directly.

### Connecting to the database

In [1]:
from experimentlog import ExperimentLog, np_to_str, str_to_np
import numpy as np
## open a connection to a database; will be created if it does not exist.
# here we use a memory database so the results are not stored to disk
e = ExperimentLog(":memory:")

### Setting up the database
When a log is set up for the first time, the database needs to be configured for the experimental sessions. 

Each sensor/information **stream** needs to be registered with the database. This could be individual sensors like a mouse (x,y) time series, or questionnaire results.
Every entry in the log stream maintains the time, tag and a "valid" flag for each entry, plus the JSON representing the data logged.

In [2]:
# check if we've already set everything up
if e.get_stage()=="init":
    
    # the mouse time series, 
    e.register_stream(name="mouse", description="A time series of x,y cursor positions",
                   # the data is optional, and can contain anything you want 
                  data={
                    "sample_rate": 60,
                    "dpi": 3000,
                    "mouse_device":"Logitech MX600"})
    
    # a matrix of some kind
    e.register_stream(name="important_matrix", description="A matrix")
    
    # and a post-condition questionnaire
    e.register_stream(name="satisfaction", 
                   description="A simple satisfaction score",
                   # here, we store the questions used for future reference
                  data={
                    "questions":["How satisfied were you with your performance?",
                                "How satisfied were you with the interface?"]}
                    )


## Sessions
**ExperimentLog** uses the concept of *sessions* to manage experimental data. Sessions are much like folders in a filesystem and usually form a hierarchy, for example:
    
    /
        Experiment1/
            ConditionA/
                Rep1/
                Rep2/
                Rep3/
            ConditionB/
                Rep1/
                Rep2/
                Rep3/
                
        Experiment 2
            ConditionA/
                Rep1/
                Rep2/
                Rep3/
                Rep4/
            ConditionC/
                Rep1/
                Rep2/
                Rep3/
                Rep4/
    

Each *session* type has **metadata** attached to it; for example giving the parameters for a given condition. 

When an experiment is run, **instances** of sessions are created, like files inside the filesystem.

In [3]:
if e.get_stage()=="init":
    # We'll register an experiment, with three different conditions
    e.register_session("Experiment", description="The main experiment", 
                           data={"target_size":40.0, "cursor_size":5.0})
    e.register_session("ConditionA",description="Condition A:circular targets", 
                           data={"targets":["circle"]})
    e.register_session("ConditionB", description="Condition B:square targets", 
                           data={"targets":["square"]})
    e.register_session("ConditionC", description="Condition C:mixed targets", 
                           data={"targets":["circle","mixed"]})


We'd usually only want to do this once-ever; this setup procedure can be recorded by changing the database **stage**:

In [4]:
# mark the database as ready to log data
e.set_stage("setup")

## Users
Each instance of a session (usually) involves experimental subjects. Each user should be registered, and then attached to a recording session. Multiple users can be attached to one session (e.g. for experiments with groups) but normally there will just be one user.

The `pseudo` module can generate pronounceable, random, verifiable pseudonyms for subjects.


In [5]:
import pseudo
user = pseudo.get_pseudo()
print user

NAHUR-ULELU


In [6]:
# now register the user with the database
e.register_user(name=user, user_vars={"age":30, "leftright":"right"})


'NAHUR-ULELU'

## Runs
Finally, each **run** of the experimental session is logged. If there are any variables that change on a per-run basis (e.g. calibration parameters) they can be stored here.

The experimenter running this trial should be specified for each run.

In [7]:
e.start_run(experimenter="JHW")

In [8]:
# attach the user to this experimental run
e.add_active_user(user)
# enter conditionA of experiment
e.enter_session("Experiment")
e.enter_session("ConditionA")
e.enter_session() # Unnammed sessions are assumed to repetitions

In [9]:
print e.session_path


/Experiment/ConditionA/None


In [10]:
# log some data
e.log("mouse", data={"x":0, "y":10})
e.log("mouse", data={"x":0, "y":20})

2

Test how fast we can write into the database:

In [11]:
%%timeit -n 50000
e.log("mouse", data={"x":20, "y":20})

50000 loops, best of 3: 9.23 µs per loop


In [12]:
# log questionnaire output
e.log("satisfaction", data={"q1":4,"q2":5})

150003

In [13]:
# leave this repetition
e.leave_session() 

# move out of condition A
e.leave_session()

In [14]:
e.enter_session("ConditionB")

In [15]:
# could log more stuff...

In [16]:
x = np.random.uniform(-1,1,(16,16))
i = e.log("important_matrix")
# if we need to attach binary data to a log file (e.g. an image), we can do this:
# in general, it is best to avoid using blobs unless absolutely necessary
e.attach_blob(i, np_to_str({"matrix":(x)}))

In [17]:
e.leave_session()
e.leave_session()
# back to the root

In [18]:
e.end_run() # end the run

In [19]:
# print some results with raw SQL queries
mouse_log = e.cursor.execute("SELECT time, json FROM mouse", ())
print "\n".join([str(m) for m in mouse_log.fetchone()])

1455182820.34
{"y": 10, "x": 0}


In [20]:
import report

In [21]:
import IPython.nbconvert
from IPython.core.display import HTML
def md_html(md):    
    return HTML(IPython.nbconvert.filters.markdown.markdown2html(md))

c:\conda64\lib\site-packages\IPython\nbconvert.py:13: ShimWarning: The `IPython.nbconvert` package has been deprecated. You should import from ipython_nbconvert instead.
  "You should import from ipython_nbconvert instead.", ShimWarning)


In [22]:
md_html(report.string_report(e.cursor))

### Post-processing
Once all data is logged, it is wise to add indices so that logs can be accessed quickly.

In [23]:
# should only do this when all data is logged; otherwise there may be
# a performance penalty
e.add_indices()

## SQL format
There are a few basic tables in the ExperimentLog:

#### Metadata
    meta: 
        id, Unique ID
        mtype,    Type of this metadata: one of LOG, SESSION, USER, PATH
        name,     Name of the object, e.g. user pseudonyn
        type,     (Optional) type tag
        description, (Optional) text description
        json       (Optional) JSON string holding any other metadata.

The metadata for a log, session or user, path. `mtype` specifies the kind of metadata it is. There are convenience views of this table:

    log_stream
    users
    session_meta
    paths

All have the same fields as above.

#### Session

        session: 
            id,          Unique ID
            start_time,  Time this session was started
            end_time,    Time this session was completed (if it was)
            last_time,   Last time a log was written for this session
            test_run,    If this is a test run or not
            random_seed, Random seed used for this session can be stored here
            valid,       If this session was marked valid or not
            complete,    If this session was marked completed or not
            parent,      ID of the session this session is a subsession of
            path,        ID of the full path this session belongs to
            json,        Any additional metadata

        
       run_session: (maps sessions to runs)
           id,           Unique ID
           run,          ID of the run
           session,      ID of the session
           
       user_session: (maps users to sessions)
           id,           Unique ID
           session,      ID of the session
           user,         ID of the user
           role,         (optional) String giving the role the user plays
           json,         (optional) Any session-specific user variables.
           
#### Logs

    log:
        id,         Unique ID
        time,       Timestamp
        valid,      Valid flag for this data (e.g. to mark faulty sensor data)
        stream,     ID of the stream this log belongs to
        session,    ID of the session this log entry belongs to
        json,       The log entry itself        
        tag,        (optional) tag for this log entry
        
        
When a new log is created, it also creates a new view with the given stream name; so `e.register_log("mouse")` creates a new view table `mouse` with the same fields as `log`.


    blob:
        id,         Unique ID
        blob,       Binary blob 
        log,        Log this blob attaches to
        

### Misc

    children: (maps sessions to all of their children sessions)
        id,     Unique ID
        parent, ID of the parent session
        child,  ID of the child session

This is automatically filled in by `enter_session()`/`leave_session()`
       
    setup:
        id,      Unique ID
        time,    Time of the stage change
        stage,   Stage of the database

This is normally only used to record whether the database is fully setup or not.



### Custom tables
If you want to log values with a **custom** table where the fields are not just plain JSON, you can add a new table to the database and just attach it to the log fields. The `log()` function returns the ID of the new log entry; use this as a foreign key in the new log table.

Example:




In [24]:
# make the new table -- must have a reference to the main
# log table
e.execute("""CREATE TABLE accelerometer 
          (id INTEGER PRIMARY KEY, device INT, x REAL, y REAL, z REAL, log INT,
          FOREIGN KEY(log) REFERENCES log(id))
          """)

# register a new stream
e.register_stream(name="acc", description="A time series of accelerometer values")

# now log a new value, put it into the separate accelerometer table and link
# it to the main log
def log_acc(dev,x,y,z):
    log_id = e.log("acc")
    e.execute("INSERT INTO accelerometer VALUES (?,?,?,?,?)", 
              (dev, x, y, z, log_id))


### Sync points
If you are recording media alongside an experimental trial (e.g. a video), you can use **sync_points** to record the link between external media and the master log.

Usage:
    
    # look up a log entry with a sync point
    t = e.execute('SELECT time FROM log WHERE tag="video_sync_mark"').fetchone()[0]
    sync("myvideo.mp4", start_time=t)
    
If you want to record a segment of a video as being aligned:

    t = e.execute('SELECT time FROM log WHERE tag="video_sync_mark"').fetchone()[0]
    # marks a synchronisation of myvideo.mp4, from 20.0 -> 25.0 to the log time starting at t
    sync("myvideo.mp4", start_time=t, duration=5.0, media_start_time=20.0)
    
    


### NTP
To ensure logs are consistent in their timings, `ExperimentLog` will try and sync to an NTP server on start-up and will record all times with the estimated clock offset already applied.

If you pass `ntp_sync=False` to the ExperimentLog constructor, this will be skipped. Custom NTP servers can also be passed as a list:

    # don't sync to NTP (not recommended)
    e = ExperimentLog(ntp_sync=False)
    
    # use custom NTP servers
    e = ExperimentLog(ntp_servers=["1.pool.ntp.org", "2.pool.ntp.org"])


### Whole-dataset metadata
Dataset-wide metadata can be set with `set_meta()` and read with `get_meta()`. The report auto-generator can use this to build automatic readme files suitable for deposit for open access.

The following fields should be set:

* *title* (title of this dataset)
* *institution* (institution(s) this dataset was recorded by)
* *authors* (comma separated list of authors)
* *license* (e.g. CC-BY-SA 2.0)
* *confidential* (e.g. No, InternalOnly, ConsortiumOnly, Confidential, StrictlyConfidential), etc.
* *funder* (name of funder and project name/code)
* *ethics* (ethics board approval number)
* *paper* (full details of associated paper)
* *short_description* (one sentence description of the data set)
* *description* (longer description of the dataset)
* *doi* (DOI of this dataset)


In [25]:
e.set_meta(title="TestSet-1", institution="University of Glasgow")
e.set_meta(authors="John Williamson", license="CC-BY-SA 2.0", confidential="no", funder="XXX Grant ABC:nnnn", ethics="CSExxxxx")
e.set_meta(paper="'A good paper', Williamson J., Proceedings of Things International 2016, pp.44-46")
e.set_meta(description="A study of the experimental logging process. Includes numerous repetitive examples of simple logged data.")
e.set_meta(short_description="A quick logging test.")
e.set_meta(doi="DOI:xxxxxx")


In [26]:
print e.get_meta()

{u'ethics': u'CSExxxxx', u'doi': u'DOI:xxxxxx', u'confidential': u'no', u'license': u'CC-BY-SA 2.0', u'title': u'TestSet-1', u'paper': u"'A good paper', Williamson J., Proceedings of Things International 2016, pp.44-46", u'authors': u'John Williamson', u'short_description': u'A quick logging test.', u'funder': u'XXX Grant ABC:nnnn', u'institution': u'University of Glasgow', u'description': u'A study of the experimental logging process. Includes numerous repetitive examples of simple logged data.'}


In [27]:
md_html(report.string_readme(e.cursor))